In [ ]:
%%loadFromPOM
<repository>
    <id>bintray-ruivieira-maven</id>
    <url>https://dl.bintray.com/ruivieira/maven</url>
</repository>

<dependency>
    <groupId>org.ruivieira.ml</groupId>
    <artifactId>als</artifactId>
    <version>0.0.1</version>
</dependency>